In [ ]:
dataset_path = "../../dataset/GOLD_XYZ_OSC_POSITIVE_COMBINED_NEW.hdf5"

In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import h5py
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, TensorDataset
import os, time
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from ray import tune
from ray.train import Checkpoint, get_checkpoint, RunConfig
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ray.tune import CLIReporter
import ray
from sklearn.model_selection import train_test_split

In [ ]:
with h5py.File(dataset_path, 'r') as f:
    X = f['X'][:]
    Y = f['Y'][:]
    Z = f['Z'][:]


In [ ]:
import gc
gc.collect()

In [ ]:

epsilon = 1e-8  # A small value to avoid division by zero
X_min = np.min(X, axis=1, keepdims=True)
X_max = np.max(X, axis=1, keepdims=True)

# Compute range
X_range = X_max - X_min

# Create a mask where range is 0
mask = X_range == 0

# Normalize to [-1, 1] (Avoid division by zero by adding epsilon)
X = np.where(mask, 0, 2 * (X - X_min) / (X_range + epsilon) - 1)


In [ ]:
modulation_schemes = range(10)
snr_levels = np.append(np.arange(0, 31, 2), 9999)

In [ ]:
# Plot constellation diagrams for the extracted dataset
def plot_constellation_snr_levels(modulation_class_index, extracted_classes, X, Y, Z):
        # Get modulation class name from extracted_classes
        class_name = extracted_classes[modulation_class_index]

        # Find indices of the target modulation class
        selected_class_indices = np.where(Y == modulation_class_index)[0]

        # Get unique SNR levels
        snr_values = np.unique(Z)

        # Plot constellation diagrams for all SNR levels
        plt.figure(figsize=(16, 20))
        for i, snr in enumerate(snr_values):
            # Find indices corresponding to the current SNR level
            snr_indices = selected_class_indices[np.where(Z[selected_class_indices] == snr)[0]]

            if len(snr_indices) == 0:
                continue

            # Extract the first sample for the current SNR level
            iq_samples = X[snr_indices[0]]
            # print(iq_samples.shape)
            # break
            # Separate into in-phase (I) and quadrature (Q) components
            I = iq_samples[:, 0].flatten()
            Q = iq_samples[:, 1].flatten()

            # Plot the scatter plot
            ax = plt.subplot(5, 4, i + 1)
            ax.scatter(I, Q, s=5, alpha=0.8, color='black')
            # ax.set_title(f"SNR: {snr}dB", fontsize=10, fontweight='bold')
            plt.text(0.5, -0.1, f"SNR: {int(snr)}dB", fontsize=20, fontweight='bold', transform=ax.transAxes, ha='center', va='center')
            # ax.axis('off')
            ax.grid(True)


        plt.tight_layout()
        plt.subplots_adjust(top=0.95)
        # plt.savefig(f"./visualization/constellation{classes[modulation_class_index]}.pdf", format="pdf")
        plt.show()


# Paths to extracted dataset and relevant parameters
classes = [
    'BPSK', 'QPSK', '8PSK', '16QAM', '64QAM', 'AM-DSB-SC', 'AM-SSB-SC', 'FM', 'GMSK', 'GFSK'
]

In [ ]:
for modulation in range(10):
    plot_constellation_snr_levels(modulation, classes, X, Y, Z)

In [ ]:

test_size = 0.1
valid_size = 0.1

train_indices = []
validation_indices = []
test_indices = []

for modulation_scheme in modulation_schemes:
    indices = np.where(Y == modulation_scheme)[0]
    # now for these indices find each SNR indices in snr_levels
    for snr in snr_levels:
        snr_indices = np.where(Z[indices] == snr)[0]

        # split into train+valid and test first
        train_valid, test = train_test_split(
            snr_indices, test_size=test_size, stratify=Z[indices][snr_indices], random_state=42
        )

        # further split train_valid into train and validation sets
        train, valid = train_test_split(
            train_valid, test_size=valid_size / (1 - test_size), stratify=Z[indices][train_valid], random_state=42
        )

        train_indices.extend(indices[train])
        validation_indices.extend(indices[valid])
        test_indices.extend(indices[test])

# convert lists to numpy arrays for shuffling
train_indices = np.array(train_indices)
validation_indices = np.array(validation_indices)
test_indices = np.array(test_indices)

# shuffle the indices
np.random.shuffle(train_indices)
np.random.shuffle(validation_indices)
np.random.shuffle(test_indices)


print("Train size: ", len(train_indices))
print("Validation size: ", len(validation_indices))
print("Test size: ", len(test_indices))

print(train_indices[:10])
print(validation_indices[:10])
print(test_indices[:10])


In [ ]:

def windowing_function(input_data):

    if input_data.shape != (1024, 2):
        raise ValueError("Input data must have shape (1024, 2).")

    window_size = 224
    overlap = window_size // 2
    step_size = window_size - overlap
    output_sequences = []

    for i in range(0, 1024 - window_size, step_size):
        output_sequences.append(input_data[i:i + window_size])

    return np.array(output_sequences)



In [ ]:
def process_indices(indices):
    X_seq = []
    Y_seq = []

    for idx in indices:
        seq = windowing_function(X[idx])
        X_seq.append(seq)
        Y_seq.append(np.full(len(seq), Y[idx], dtype=np.float16))

    return np.concatenate(X_seq, axis=0, dtype=np.float16), np.concatenate(Y_seq, axis=0)

# Process training, validation, and test sets
X_train, Y_train = process_indices(train_indices)
X_valid, Y_valid = process_indices(validation_indices)
X_test, Y_test = process_indices(test_indices)

In [ ]:
import gc
del X, Y, Z 
gc.collect()

In [ ]:


def amplitude_phase_seq_batch(iq_seq):
    amplitude = np.sqrt(np.sum(np.square(iq_seq), axis=2, keepdims=True))
    phase = np.arctan2(iq_seq[:, :, 1], iq_seq[:, :, 0])[..., np.newaxis]
    return np.concatenate([iq_seq, amplitude, phase], axis=2)

# def amplitude_phase_seq_batch(iq_seq):
#     # Precompute squared values to avoid recalculating them
#     squared = np.square(iq_seq)

#     # Calculate amplitude and phase efficiently
#     amplitude = np.sqrt(np.sum(squared, axis=2, keepdims=True))
#     phase = np.arctan2(iq_seq[..., 1], iq_seq[..., 0])[..., np.newaxis]

#     # Concatenate once instead of multiple times
#     return np.concatenate((iq_seq, amplitude, phase), axis=2)


In [ ]:

def data_preparation_alexnet(input_data, device):
    iq_channel = input_data[:, :, :2] # Shape: (32, 224, 2)
    amp_channel = input_data[:, :, 2:3]  # Shape: (32, 224, 1)
    phase_channel = input_data[:, :, 3:]  # Shape: (32, 224, 1)

    # print(iq_channel.shape)
    iq_channel = iq_channel.repeat(1, 1, 112)
    amp_channel = amp_channel.repeat(1, 1, 224)
    phase_channel = phase_channel.repeat(1, 1, 224)

    # adding new dimension for channel to make the shape (8, 3, 224, 224)
    input_tensor = torch.stack([iq_channel, amp_channel, phase_channel], dim=1)

    input_tensor = input_tensor.to(device)
    return input_tensor


def data_preparation(initial_batch_data, device):
    iq_channel = initial_batch_data[:, :, :2]  # Shape: (32, 224, 2)
    amp_channel = initial_batch_data[:, :, 2:3]  # Shape: (32, 224, 1)
    phase_channel = initial_batch_data[:, :, 3:]  # Shape: (32, 224, 1)

    iq_channel = iq_channel.repeat(1, 1, 112)
    amp_channel = amp_channel.repeat(1, 1, 224)
    phase_channel = phase_channel.repeat(1, 1, 224)

    input_tensor = torch.stack([iq_channel, amp_channel, phase_channel], dim=1)
    input_tensor = input_tensor.to(device)
    return input_tensor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class AlexNetModified(nn.Module):
    def __init__(self):
        super(AlexNetModified, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding = 0),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True)
        )
        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pooling(x)
        # x = torch.flatten(x, 1)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # self.fc1 = nn.Linear(hidden_size * 8, 512)
        # self.fc2 = nn.Linear(512, 256)
        # self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device).detach()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device).detach()
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.size(0), -1)  # Flatten all hidden states
        # out = F.relu(self.fc1(out))
        # out = F.relu(self.fc2(out))
        # out = self.fc3(out)
        return out




class FullyConnectedLayer(nn.Module):
    def __init__(self, input_size, l_1, l_2, drop_factor, output_size):
        super(FullyConnectedLayer, self).__init__()
        self.l1 = nn.Linear(input_size, l_1)
        self.dropout1 = nn.Dropout(drop_factor)
        self.l2 = nn.Linear(l_1, l_2)
        self.dropout2 = nn.Dropout(drop_factor)
        self.l3 = nn.Linear(l_2, output_size)

    def forward(self, x):
        x = F.leaky_relu(self.l1(x))
        x = self.dropout1(x)
        x = F.leaky_relu(self.l2(x))
        x = self.dropout2(x)
        x = self.l3(x)
        return x

In [ ]:
params = {
    "input_size": 256,
    "hidden_size": 256,
    "num_layers": 2,
    "num_classes": 9,
    "num_epochs": 5,
    "batch_size": 32,
    "num_seq": 8,
    "log_interval": 10,
    "l_1": 512,
    "l_2": 256,
    "drop_factor": 0.5,
    "output_size": 9
}

In [ ]:
class CustomDataloader(DataLoader):
    def __init__(self, dataset, batch_size, num_seq, shuffle=True):
        self.X = dataset.tensors[0]  # Input data
        self.Y = dataset.tensors[1]  # Labels
        self.batch_size = batch_size*num_seq
        self.num_seq = num_seq
        self.shuffle = shuffle
        self.indices = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indices)


    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if self.i >= len(self.X):
            raise StopIteration

        # Get indices for the batch
        indices = self.indices[self.i:self.i + self.batch_size]
        self.i += self.batch_size

        batch_X = []
        batch_Y = []

        for index in indices:
            seq = self.X[index]  # Get the sequence data
            batch_X.append(seq)

            # Only add corresponding labels for the sequences indexed by multiple of `num_seq`
            if index % self.num_seq == 0:
                batch_Y.append(self.Y[index])

        # Stack the sequences into one tensor for X and Y
        # Shape: [batch_size, seq_len, 224, 4]
        batch_X_tensor = torch.stack(batch_X)
        # Shape: [batch_size // num_seq, label_dim]
        batch_Y_tensor = torch.stack(batch_Y)

        # Move tensors to the correct device
        return batch_X_tensor.to(device), batch_Y_tensor.to(device)

    def __len__(self):
        return len(self.X) // self.batch_size

In [ ]:
def train_model(config, train_dataloader, valid_dataloader,checkpoint_dir=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize models with tuned hyperparameters
    ALEX_model = AlexNetModified().to(device)
    LSTM_model = LSTMModel(
        input_size=params['input_size'], hidden_size=params['hidden_size'], num_layers=2).to(device)
    FC_model = FullyConnectedLayer(
        input_size=params['hidden_size'] * params['num_seq'], l_1=config["l_1"], l_2=config["l_2"], drop_factor=config["drop_factor"], output_size=params['output_size']
    ).to(device)

    # Define criterion and optimizer with the tuned learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        list(ALEX_model.parameters()) +
        list(LSTM_model.parameters()) + list(FC_model.parameters()),
        lr=config["learning_rate"],
    )

    # Load checkpoint if available
    if checkpoint_dir:
        checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pt")
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, weights_only=True)
            ALEX_model.load_state_dict(checkpoint["ALEX_model_state_dict"])
            LSTM_model.load_state_dict(checkpoint["LSTM_model_state_dict"])
            FC_model.load_state_dict(checkpoint["FC_model_state_dict"])
            optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
            start_epoch = checkpoint["epoch"]
        else:
            start_epoch = 0
    else:
        start_epoch = 0

    for epoch in range(start_epoch, params["num_epochs"]):
        ALEX_model.train()
        LSTM_model.train()
        FC_model.train()

        running_loss = 0.0
        start_time = time.time()

        for batch_idx, (inputs, labels) in enumerate(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            # print(inputs.shape)
            alexnet_input = data_preparation_alexnet(inputs, device)
            alexnet_input = alexnet_input.view(-1, 8, 3, 224, 224)
            outputs_list = []

            # Forward pass through AlexNet
            for i in range(alexnet_input.shape[0]):
                alexnet_output = ALEX_model(alexnet_input[i])
                alexnet_output = alexnet_output.view(8, 256)
                outputs_list.append(alexnet_output)

            lstm_input = torch.stack(outputs_list)
            lstm_output = LSTM_model(lstm_input)
            fc_output = FC_model(lstm_output)

            loss = criterion(fc_output, labels.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % params["log_interval"] == 0:
                print(f"Epoch [{epoch+1}/{params['num_epochs']}], Batch [{batch_idx}/{len(train_dataloader)}], Loss: {loss.item():.4f}", flush=True)

        # Average train loss for epoch
        avg_train_loss = running_loss / len(train_dataloader)

        # Validation phase
        ALEX_model.eval()
        LSTM_model.eval()
        FC_model.eval()

        val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_labels in valid_dataloader:
                val_inputs, val_labels = val_inputs.to(
                    device), val_labels.to(device)
                val_inputs = data_preparation_alexnet(val_inputs, device)
                val_inputs = val_inputs.view(-1, 8, 3, 224, 224)
                val_outputs_list = []

                for i in range(val_inputs.shape[0]):
                    alexnet_output = ALEX_model(val_inputs[i])
                    alexnet_output = alexnet_output.view(8, 256)
                    val_outputs_list.append(alexnet_output)

                lstm_input = torch.stack(val_outputs_list)
                lstm_output = LSTM_model(lstm_input)
                fc_output = FC_model(lstm_output)

                loss = criterion(fc_output, val_labels.long())
                val_loss += loss.item()

        avg_val_loss = val_loss / len(valid_dataloader)
        print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}")

        # Checkpoint and log loss metrics
        checkpoint_data = {
            "epoch": epoch + 1,
            "ALEX_model_state_dict": ALEX_model.state_dict(),
            "LSTM_model_state_dict": LSTM_model.state_dict(),
            "FC_model_state_dict": FC_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_loss": avg_val_loss,
        }

        checkpoint_path = os.path.join("/kaggle/working/checkpoints/", "checkpoint.pt")
        torch.save(checkpoint_data, checkpoint_path)

        ray.train.report(dict(val_loss=avg_val_loss, train_loss=avg_train_loss))
        print(f"Epoch [{epoch+1}/{params['num_epochs']}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

In [ ]:
X_train = amplitude_phase_seq_batch(X_train)
X_valid = amplitude_phase_seq_batch(X_valid)
X_test = amplitude_phase_seq_batch(X_test)

In [ ]:
# convert X_train, Y_train to tensor
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

X_valid = torch.tensor(X_valid, dtype=torch.float32).to(device)
Y_valid = torch.tensor(Y_valid, dtype=torch.float32).to(device)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = TensorDataset(X_train, Y_train)
train_dataloader = CustomDataloader(train_data, batch_size=params["batch_size"],shuffle=False, num_seq= params["num_seq"])

valid_data = TensorDataset(X_valid, Y_valid)
valid_dataloader = CustomDataloader(valid_data, batch_size=params["batch_size"], shuffle=False, num_seq=params["num_seq"])


test_data = TensorDataset(X_test, Y_test)
test_dataloader = CustomDataloader(test_data, batch_size=params["batch_size"], shuffle=False, num_seq=params["num_seq"])



In [ ]:

# Define the search space
search_space = {
    "learning_rate": tune.loguniform(5e-5, 3e-4),
    "drop_factor": tune.grid_search([0.5, 0.6, 0.7]),
    "l_2": tune.grid_search([256, 128]),
    "l_1": tune.grid_search([512, 256])
}

# Define the ASHA scheduler
scheduler = ASHAScheduler(
    metric="val_loss", mode="min", max_t=params["num_epochs"], grace_period=1, reduction_factor=2
)

# Define the reporter
reporter = CLIReporter(
    metric_columns=["val_loss", "train_loss", "training_iteration"])


In [ ]:

checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs("ray_results", exist_ok=True)

# Run Ray Tune with defined search space
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(
            train_model, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader, checkpoint_dir=checkpoint_dir
        ),
        resources={"cpu": 4, "gpu": 1 if torch.cuda.is_available() else 0},
    ),
    param_space=search_space,
    tune_config=tune.TuneConfig(scheduler=scheduler),
    run_config=RunConfig(storage_path="./ray_results")
)

results = tuner.fit()

In [ ]:
print("Best hyperparameters found were: ", results.get_best_result("val_loss", mode="min").config)